# Final Project （part 2）
Team leader: Zhaoliang Zheng

Team member: Anqi Shen, Enbo Yu


In [3]:
import os
ROOTDIR = os.path.abspath(os.path.dirname('__file__'))
print(ROOTDIR)

C:\Users\zzl\data\final project


In [4]:
import pandas as pd
#Load dataset and check it
df = pd.read_csv(os.path.join(ROOTDIR, 'fake_or_real_news.csv'))

## Step1: DATA PREPROCESSING

In [5]:
#define a binarize function and binrize the label
def binarize(word):
    if word== 'REAL':
        return 0
    else:
        return 1
df.label= df.label.apply(binarize)

In [6]:
#in this step, I'm gonna drop the first column which is meaningless here
df.drop('Unnamed: 0', axis=1, inplace=True)

## Step2: DATA PROCESSING:TOKENIZE & CLEAN THE TEXT

In [7]:
import nltk
from nltk.corpus import gutenberg
from gensim import models
from string import punctuation
from nltk.corpus import stopwords

D:\program files(X86)\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
#load raw title and text dataset into different varibles
texts = df.text.values
titles = df.title.values
#simply tokenize the text and title
tokenized_text = [nltk.word_tokenize(text) for text in texts]
tokenized_title = [nltk.word_tokenize(title) for title in titles]

In [9]:
sw = stopwords.words('english')
def clean_advance(tokenized_list, sw):
    new_list = []
    for doc in tokenized_list:
        new_list.append([token.lower() for token in doc if token.lower() not in sw
                        and token.lower() != "“" and token.lower() != "”"
                        and token.lower() != "‘" and token.lower() != "’"
                        and token.lower() != "``" and token.lower() != "''"
                        and token.lower() != "—"  and token.lower() != "–"
                        and token.lower() !="," and token.lower() != "."
                        and token.lower() != ":" and token.lower() != "?"
                        and token.lower() != "(" and token.lower() != ")"
                        and token.lower() != "'s" and token.lower() != "n't"
                        and token.lower() != "'d" and token.lower() != "'ve"
                        and token.lower() != "!" and token.lower() != "'"
                        and token.lower() != "&" and token.lower() != "*"
                        and token.lower() != "..." and token.lower() != "…" 
                        and token.lower() != "#" and token.lower() != "-"
                        and token.lower() != "[" and token.lower() != "]" 
                        and token.lower() != "%" and token.lower() !="|" 
                        and token.lower() != ";" ])
    return new_list

In [10]:
# Remove punctuations and stopwords,lower-case and other useless mark from tokenized title and text
cleaned_title = clean_advance(tokenized_title, sw)
cleaned_text = clean_advance(tokenized_text, sw)

In [11]:
from collections import Counter
token_counter = Counter(token.lower() for sentence in cleaned_text for token in sentence)
top20 = token_counter.most_common()[:20]
for i, t in enumerate(top20):
    print('{:>2}.{:>11}  freq: {:>7}'.format(i+1, t[0], t[1]))

 1.      trump  freq:   21906
 2.       said  freq:   21162
 3.    clinton  freq:   17237
 4.      would  freq:   12928
 5.     people  freq:   11598
 6.        one  freq:   11295
 7.        new  freq:    9263
 8.      state  freq:    8744
 9.  president  freq:    8509
10.       also  freq:    8206
11.      obama  freq:    8121
12.   campaign  freq:    7642
13.         us  freq:    7511
14.    hillary  freq:    7127
15.       like  freq:    7038
16.      could  freq:    6705
17.       time  freq:    6432
18.       even  freq:    6424
19.     states  freq:    6142
20.       many  freq:    5847


### 1way：Extend title with text

In [12]:
import copy
c_extend=copy.deepcopy(cleaned_title)
for i in range(texts.shape[0]):
   c_extend[i].extend(cleaned_text[i])

In [13]:
type(c_extend)

list

### 2way：Put title with text into two varibles and weight them

In [14]:
c_title=copy.deepcopy(cleaned_title)
c_text=copy.deepcopy(cleaned_title)

In [15]:
c_title[0]

['smell', 'hillary', 'fear']

## Step3: Feature extration & vecterize data

### method1: TF-IDF

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
#x_extend = vec.fit_transform(c_extend)
#x_title = vec.fit_transform(c_title)
#x_text = vec.fit_transform(c_text)

In [80]:
vec_str = []
for words in c_extend:
    str = ''
    for i in words:
        str += i
        str += ' '
    vec_str.append(str)

In [81]:
vectorizer.fit(vec_str)
tfidf_extend = vectorizer.transform(vec_str)
tfidf_m=tfidf_extend.toarray()

In [82]:
print(tfidf_m.shape,tfidf_extend.shape)

(6335, 67918) (6335, 67918)


disadvantages:
- vector sizes become huge, equal to vocabulary size
    - sparsity
    - curse of dimensionality
    - computationally expensive
    

### Gaussian Naive Bayes

Perhaps the easiest naive Bayes classifier to understand is Gaussian naive Bayes. 

Of course, the final classification will only be as good as the model assumptions that lead to it, which is why Gaussian naive Bayes often does not produce very good results. Still, in many cases—especially as the number of features becomes large—this assumption is not detrimental enough to prevent Gaussian naive Bayes from being a useful method.

### Multinomial Naive Bayes
The Gaussian assumption just described is by no means the only simple assumption that could be used to specify the generative distribution for each label. Another useful example is multinomial naive Bayes, where the features are assumed to be generated from a simple multinomial distribution. The multinomial distribution describes the probability of observing counts among a number of categories, and thus multinomial naive Bayes is most appropriate for features that represent counts or count rates.

The idea is precisely the same as before, except that instead of modeling the data distribution with the best-fit Gaussian, we model the data distribuiton with a best-fit multinomial distribution.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

modelNB = make_pipeline(TfidfVectorizer(), MultinomialNB())

### When to Use Naive Bayes
- Because naive Bayesian classifiers make such stringent assumptions about data, they will generally not perform as well as a more complicated model. That said, they have several advantages:
    - They are extremely fast for both training and prediction
    - They provide straightforward probabilistic prediction
    - They are often very easily interpretable
    - They have very few (if any) tunable parameters

In [170]:
from gensim import corpora
# Create a dictionary from list of documents
dictionary = corpora.Dictionary(c_text)

In [171]:
corpus = [dictionary.doc2bow(text) for text in c_text]
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)

In [172]:
n = 10
print('Example review featurized with TF-IDF scores : \n{}'.format([(dictionary[i[0]], round(i[1],3)) for i in tfidf[corpus[n]]]))

Example review featurized with TF-IDF scores : 
[('hillary', 0.078), ('political', 0.021), ('bernie', 0.044), ('supporters', 0.072), ('tried', 0.044), ('clinton', 0.295), ('campaign', 0.137), ('clintons', 0.062), ('eluded', 0.126), ('fire', 0.101), ('glimpse', 0.092), ('iowa', 0.34), ('three', 0.029), ('even', 0.03), ('news', 0.042), ('lead', 0.037), ('bill', 0.101), ('need', 0.053), ('--', 0.074), ('sanders', 0.043), ('ever', 0.033), ('hope', 0.041), ('interview', 0.04), ('near', 0.046), ('woman', 0.087), ('policy', 0.03), ('saturday', 0.099), ('right', 0.023), ('lays', 0.095), ('added', 0.036), ('gold', 0.07), ('turn', 0.041), ('plan', 0.039), ('wife', 0.157), ('voters', 0.03), ('help', 0.03), ('two', 0.017), ('days', 0.088), ('president', 0.031), ('book', 0.099), ('old', 0.045), ('one', 0.019), ('cnn', 0.04), ('latest', 0.044), ('run', 0.035), ('show', 0.032), ('work', 0.027), ('years', 0.037), ('long', 0.027), ('democrats', 0.032), ('deliver', 0.066), ('different', 0.035), ('instea

In [181]:
feature_importance = []
w_tfidf= []
for i in range(texts.shape[0]):
    for j in tfidf[corpus[i]]:
        if round(j[1],3) > 0.5 and round(j[1],3) < 0.6:
             feature_importance.append(dictionary[j[0]])
             w_tfidf.append(round(j[1],3))
#important_words = list(set(important_words))
print(type(feature_importance),len(feature_importance))
print(type(w_tfidf),len(w_tfidf))

<class 'list'> 1389
<class 'list'> 1389


In [182]:
for i in range(len(w_tfidf)):
    print('feature word: {} ,         TF-IDF scores: {} '.format(feature_importance[i],w_tfidf[i]))

feature word: fbi ,         TF-IDF scores: 0.59 
feature word: ryan ,         TF-IDF scores: 0.566 
feature word: va ,         TF-IDF scores: 0.514 
feature word: picture ,         TF-IDF scores: 0.511 
feature word: sanctions ,         TF-IDF scores: 0.502 
feature word: metabiology ,         TF-IDF scores: 0.526 
feature word: muscles ,         TF-IDF scores: 0.591 
feature word: c ,         TF-IDF scores: 0.56 
feature word: hindu ,         TF-IDF scores: 0.521 
feature word: weiner ,         TF-IDF scores: 0.503 
feature word: hike ,         TF-IDF scores: 0.509 
feature word: afghanistan ,         TF-IDF scores: 0.54 
feature word: abuse ,         TF-IDF scores: 0.531 
feature word: substance ,         TF-IDF scores: 0.592 
feature word: crops ,         TF-IDF scores: 0.505 
feature word: facebook ,         TF-IDF scores: 0.539 
feature word: hispanic ,         TF-IDF scores: 0.528 
feature word: hispanics ,         TF-IDF scores: 0.502 
feature word: laconia ,         TF-IDF scor

In [183]:
feature_text = np.zeros((texts.shape[0], len(feature_importance)))
print(type(feature_importance),feature_text.shape)

<class 'list'> (6335, 1389)


In [184]:
import numpy as np
text= {}
index=0
for word in feature_importance:
    text[word] = index
    index +=1

In [185]:
for i in range(texts.shape[0]):
   # for j in tfidf[corpus[i]]:
        for word in c_title[i]:
            if word in feature_importance:
                feature_text[i, text[word]] +=1
print(feature_text.shape)

(6335, 1389)


In [186]:
feature_vec = pd.DataFrame(feature_text)
x_0 = pd.concat([feature_vec], axis=1)

In [187]:
#forming training sets and test sets
from sklearn.model_selection import train_test_split

y = np.asarray(df['label'])
seed = 42
test_size = 0.15 #
validation_size = 0.15/(1-test_size)

x0_train, x0_test, y0_train, y0_test = train_test_split(x_0, y, test_size=test_size, random_state=seed)
x0_train, x0_validation, y0_train, y0_validation = train_test_split(x0_train, y0_train, test_size=validation_size, random_state=seed)
print(y0_train.shape)
print(y0_test.shape)
print(y0_validation.shape)

(4433,)
(951,)
(951,)


In [188]:
from sklearn.linear_model import LogisticRegression

model0 = LogisticRegression()
model0 = model.fit(x0_train, y0_train)

y0_pred = model.predict(x0_test)
print("LR:")
print(classification_report(y0_test, y0_pred, target_names=target_names))
print(model0.score(x0_test, y0_test))

LR:
             precision    recall  f1-score   support

       fake       0.81      0.68      0.74       492
       real       0.71      0.82      0.76       459

avg / total       0.76      0.75      0.75       951

0.750788643533123


In [192]:
print(x.shape,feature_text.shape)

(6335, 300) (6335, 1389)


In [193]:
import numpy as np
A = np.hstack((x, feature_text))
print(A.shape)

(6335, 1689)


In [194]:
#forming training sets and test sets
from sklearn.model_selection import train_test_split

y = np.asarray(df['label'])
seed = 42
test_size = 0.15 #
validation_size = 0.15/(1-test_size)

x1_train, x1_test, y1_train, y1_test = train_test_split(A, y, test_size=test_size, random_state=seed)
x1_train, x1_validation, y1_train, y1_validation = train_test_split(x1_train, y1_train, test_size=validation_size, random_state=seed)
print(y1_train.shape)
print(y1_test.shape)
print(y1_validation.shape)

(4433,)
(951,)
(951,)


In [195]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression()
model1 = model.fit(x1_train, y1_train)

y1_pred = model.predict(x1_test)
print("LR:")
print(classification_report(y1_test, y1_pred, target_names=target_names))
print(model0.score(x1_test, y1_test))

LR:
             precision    recall  f1-score   support

       fake       0.94      0.94      0.94       492
       real       0.93      0.93      0.93       459

avg / total       0.93      0.93      0.93       951

0.9348054679284963


In [208]:
from sklearn.svm import SVC
svc = SVC()
target_names = ['fake', 'real']

svc.fit(x1_train, y1_train)

y_2pred = svc.predict(x1_test)
print("SVM:")
print(classification_report(y1_test, y_2pred, target_names=target_names))
print(svc.score(x1_test, y1_test))

SVM:
             precision    recall  f1-score   support

       fake       0.93      0.91      0.92       492
       real       0.91      0.92      0.91       459

avg / total       0.92      0.92      0.92       951

0.9169295478443743


In [196]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.0,
        subsample=0.6,
        colsample_bytree=0.8,
        reg_alpha=0.0005,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb2.fit(x1_train, y1_train)
y1_pred = xgb2.predict(x1_test)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y1_test, y1_pred, target_names=target_names))
print(xgb2.score(x1_test, y1_test))

XGBoost:
             precision    recall  f1-score   support

       fake       0.94      0.94      0.94       492
       real       0.93      0.94      0.94       459

avg / total       0.94      0.94      0.94       951

0.9390115667718192


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### method2: doc2vec

In [16]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import reuters

class TagDocIterator:
    def __init__(self, doc_list, idx_list):
        self.doc_list = doc_list
        self.idx_list = idx_list
        
    def __iter__(self):
        for doc, idx, in zip(self.doc_list, self.idx_list):
            tag = [idx]
            yield TaggedDocument(words=doc, tags=tag)

##### Doc2Vec Parameters

- `dm` (distributed memory): '0' indicates DBOW model; '1' indicates DM
- `size`: Number of dimensions for the embedding model
- `window`: Number of context words to observe in each direction within a document
- `min_count`: Minimum frequency for words included in model
- `mean`: if 0 (default), use the sum of the context word vectors. If 1, use the mean. Only applies when dm is used in non-concatenative mode. 

In [17]:
def d2v_params():
    params = {
        'dm': (0, 1),
        'size': (100, 150, 300),
        'window': (3, 5, 8, 10, 15),
        'mean': (0, 1),
        'min_count': (5, 10, 20, 30), 
    }
    return params

In [18]:
# Create and train the doc2vec model by changing its 
def d2v_fit(dm,size,window,mean,min_count):
  doc2vec = Doc2Vec(size=size, 
                    window=window,
                    min_count=min_count,
                    dm = dm,
                    mean=mean,
                    alpha = 0.05,
                    iter=10)
  # Build the word2vec model from the corpus
  doc2vec.build_vocab(TagDocIterator(c_extend, df.index))
  doc2vec.train(TagDocIterator(c_extend, df.index), epochs = 5, total_examples = doc2vec.corpus_count)
  return doc2vec.docvecs

In [87]:
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

y = np.asarray(df['label'])
seed = 42
test_size = 0.2
best_ac=0.0
b_dm=[]
b_size=[]
b_window=[]
b_mean=[]
b_min_count=[]
for dm,size,window,mean,min_count in product( d2v_params()['dm'],d2v_params()['size'], d2v_params()['window'], d2v_params()['mean'],d2v_params()['min_count']):
        docvecs = d2v_fit(dm,size,window,mean,min_count)
        x = np.empty([df.shape[0],size])
        for i in range(df.shape[0]):
            x[i] = docvecs[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)
        model = LogisticRegression()
        model = model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        accuracy= model.score(x_test, y_test)
        if best_ac < accuracy:
            best_ac=accuracy
            b_dm=dm
            b_size=size
            b_window=window
            b_mean=mean
            b_min_count=min_count
            print('dm: {} , size: {} , window: {} , mean: {} , min_count: {} , accuracy: {}'.format(b_dm,b_size,b_window,b_mean,b_min_count, best_ac))

D:\program files(X86)\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
D:\program files(X86)\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


dm: 0 , size: 100 , window: 3 , mean: 0 , min_count: 5 , accuracy: 0.9273875295974744
dm: 0 , size: 100 , window: 3 , mean: 1 , min_count: 5 , accuracy: 0.9313338595106551
dm: 0 , size: 100 , window: 5 , mean: 0 , min_count: 10 , accuracy: 0.9329123914759274
dm: 0 , size: 100 , window: 5 , mean: 1 , min_count: 30 , accuracy: 0.9344909234411997
dm: 0 , size: 300 , window: 8 , mean: 1 , min_count: 10 , accuracy: 0.936069455406472


In [36]:
docvecs = d2v_fit(0,300,8,1,10) 

D:\program files(X86)\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
D:\program files(X86)\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [37]:
import numpy as np
x = np.empty([df.shape[0],300])
for i in range(df.shape[0]):
            x[i] = docvecs[i]

## Step 4: train, test, validation set split

ratio of spliting:

train : validation : test = 8 : 1 : 1

                            7 : 1.5: 1.5

                            7 : 1 : 2
                            
                            7 : 2 : 1
                            
                            6 : 2 : 2
                            
                            5 : 2.5: 2.5

In [38]:
#forming training sets and test sets
from sklearn.model_selection import train_test_split

y = np.asarray(df['label'])
seed = 42
test_size = 0.15 #
validation_size = 0.15/(1-test_size)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=validation_size, random_state=seed)
print(y_train.shape)
print(y_test.shape)
print(y_validation.shape)

(4433,)
(951,)
(951,)


In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(x_train, y_train)

y_pred = model.predict(x_test)
model.score(x_test, y_test)

0.9337539432176656

result：                                       accuracy                                  

train : validation : test =                  
                            
                            8 : 1 : 1        0.9290220820189274
                
                            7 : 1.5 : 1.5    0.9337539432176656
                            
                            7 : 1 : 2        0.930544593528019
                                    
                            7 : 2 : 1        0.9290220820189274 
                                    
                            6 : 2 : 2        0.9297553275453828
                                    
                            5 : 2.5: 2.5     0.9248737373737373
so we decided to take 7:1.5:1.5 as our ratio

## Step5: Try different classifiers

Let's try the different models here and see how it preforms on the validation set

*** models includes:

1:logistic regression

2:SVM

3:XGBoost

4:random forest

5:Neural network

### 1:Logistic regression

In [72]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(x_train, y_train)

y_pred = model.predict(x_validation)


In [74]:
from sklearn.metrics import classification_report
target_names = ['fake', 'real']
print('logistic regression:')
print(classification_report(y_validation, y_pred, target_names=target_names))
print(model.score(x_validation, y_validation))

logistic regression:
             precision    recall  f1-score   support

       fake       0.92      0.93      0.93       454
       real       0.94      0.93      0.93       497

avg / total       0.93      0.93      0.93       951

0.9305993690851735


### 2:SVM

In [71]:
from sklearn.svm import SVC
svc = SVC()
target_names = ['fake', 'real']

svc.fit(x_train, y_train)

y_pred = svc.predict(x_validation)
print("SVM:")
print(classification_report(y_validation, y_pred, target_names=target_names))
print(svc.score(x_validation, y_validation))

SVM:
             precision    recall  f1-score   support

       fake       0.92      0.93      0.93       454
       real       0.94      0.93      0.93       497

avg / total       0.93      0.93      0.93       951

0.9284963196635121


### 3:XGBoost

In [67]:
import time
from xgboost import XGBClassifier
start_xgb = time.clock()

XGmodel = XGBClassifier(max_depth=8, learning_rate=0.8, 
                        n_estimators=1000, silent=True, 
                        objective='binary:logistic', nthread=-1, 
                        gamma=0, min_child_weight=1, max_delta_step=0, 
                        subsample=1, colsample_bytree=1, 
                        colsample_bylevel=1, reg_alpha=0, 
                        reg_lambda=1, scale_pos_weight=0.9, 
                        base_score=0.6, seed=50, missing=None)

XGmodel.fit(x_train, y_train)

elapsed_xgb = (time.clock() - start_xgb)

print("Time used:",elapsed_xgb," s")

Time used: 45.55409682051322  s


In [69]:
y_vpred = XGmodel.predict(x_validation)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_validation, y_vpred, target_names=target_names))
print(XGmodel.score(x_validation, y_validation))

XGBoost:
             precision    recall  f1-score   support

       fake       0.92      0.94      0.93       454
       real       0.95      0.93      0.94       497

avg / total       0.94      0.94      0.94       951

0.935856992639327


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 4:Random forest

In [40]:
%%time
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 20, max_features=100, random_state=seed)
RF_model = RF_model.fit(x_train, y_train)

Wall time: 6.83 s


In [41]:
y_vpred_rf = RF_model.predict(x_validation)
target_names = ['fake', 'real']
print('random forest:')
print(classification_report(y_validation, y_vpred_rf, target_names=target_names))
print(RF_model.score(x_validation, y_validation))

random forest:
             precision    recall  f1-score   support

       fake       0.88      0.91      0.90       454
       real       0.92      0.89      0.90       497

avg / total       0.90      0.90      0.90       951

0.9011566771819137


### 5：multilayer perceptron neural network

In [77]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=300)

In [78]:
params = {
    'alpha': 0.005,
    'hidden_layer_sizes': (500, 800)
}

In [82]:
%%time
mlp.set_params(**params)
mlp.fit(x_train, y_train)
y_pred_mlp = mlp.predict(x_validation)

Wall time: 1min 12s


target_names = ['fake', 'real']
print('random forest:')
print(classification_report(y_validation, y_pred_mlp, target_names=target_names))
print(mlp.score(x_validation, y_validation))

### conclusion of this part:

- `logistic regression`: simple binary clasiifier, we use this classifier to do a lot of hypertuning on doc2vec and ratio of train, test,validation split
- `svm`: socre:0.928, Support Vector Machines is good at linear binary question so we don't descuss here.
- `XGBoost`: score:0.936, the best result we got so far, so we decided to focus on this model and carefully do hypertuning on this model.
- `Random forest`: score:0.901, the score is the worse because we just set two parameters in our model, which is estimator and max_feature, and this two paramters is set by our intuition, we haven't did hypertuning yet.
- `Neural network`: score:0.918, it's not that good because we need more information to train our neural network, but in our case, the data is still not that large  


![](compare.png)

## Step6: hypertuning

### 1:hypertuning on Random forest

random forest parameters neeeds to be tuned:

(1)max_features
- The number of features to consider while searching for a best split. These will be randomly selected.
- As a thumb-rule, square root of the total number of features works great but we should check upto 30-40% of the total number of features.
- Higher values can lead to over-fitting but depends on case to case.

(2)max_depth
- The maximum depth of a tree.
- Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.
- Should be tuned using CV.

(3)min_samples_leaf
- Defines the minimum samples (or observations) required in a terminal node or leaf.
- Used to control over-fitting similar to min_samples_split.
- Generally lower values should be chosen for imbalanced class problems because the regions in which the minority class will be in majority will be very small.

(4)min_samples_split
- Defines the minimum number of samples (or observations) which are required in a node to be considered for splitting.
- Used to control over-fitting. Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree.
- Too high values can lead to under-fitting hence, it should be tuned using CV.

![](tree-infographic.png)

In [85]:
import numpy as np
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# build a random forest classifier
clf = RandomForestClassifier(n_estimators=20)
# Utility function to report best scores
def report(grid_scores, n_top=5):
    # sort scores based on metric so we can grab the n_top models
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    # iterate over the n_top models
    for i in range(n_top):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              grid_scores['mean_test_score'][i],
              grid_scores['std_test_score'][i]))
        print("Parameters: {0}".format(grid_scores['params'][i]))
        print("")        
param_dist = {"max_depth": [3, 5, 7],
              "max_features": sp_randint(1, 100),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
# number of models we are going to train
n_iter_search = 20
# create our randomized gridsearch classifier
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,n_jobs=-1, return_train_score=True)
# start a timer so we know how long the random gridsearch took
start = time()
# perform the random gridsearch
random_search.fit(x, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
# print the top 3 model outputs from the random gridsearch
report(random_search.cv_results_)

RandomizedSearchCV took 63.60 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.901 (std: 0.003)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 7, 'max_features': 82, 'min_samples_leaf': 2, 'min_samples_split': 5}

Model with rank: 2
Mean validation score: 0.863 (std: 0.007)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 86, 'min_samples_leaf': 7, 'min_samples_split': 9}

Model with rank: 3
Mean validation score: 0.899 (std: 0.009)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 7, 'max_features': 90, 'min_samples_leaf': 4, 'min_samples_split': 7}

Model with rank: 4
Mean validation score: 0.859 (std: 0.003)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'max_features': 81, 'min_samples_leaf': 2, 'min_samples_split': 4}

Model with rank: 5
Mean validation score: 0.889 (std: 0.003)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth

In [42]:
%%time
seed=42
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 20, max_depth=7,max_features=82, min_samples_leaf= 2, min_samples_split= 5,random_state=seed)
RF_model = RF_model.fit(x_train, y_train)

Wall time: 3.18 s


In [43]:
from sklearn.metrics import classification_report
y_pred_rf = RF_model.predict(x_test)
target_names = ['fake', 'real']
print('random forest:')
print(classification_report(y_test, y_pred_rf, target_names=target_names))
print(RF_model.score(x_test, y_test))

random forest:
             precision    recall  f1-score   support

       fake       0.91      0.92      0.92       492
       real       0.92      0.90      0.91       459

avg / total       0.91      0.91      0.91       951

0.9148264984227129


### 2:hypertuning on XGBoost

#### The XGBoost Advantage

(1)Regularization:

Standard GBM implementation has no regularization like XGBoost, therefore it also helps to reduce overfitting.
In fact, XGBoost is also known as ‘regularized boosting‘ technique.

(2)Parallel Processing:

XGBoost implements parallel processing and is blazingly faster as compared to GBM.
But hang on, we know that boosting is sequential process so how can it be parallelized? We know that each tree can be built only after the previous one, so what stops us from making a tree using all cores? I hope you get where I’m coming from. Check this link out to explore further.
XGBoost also supports implementation on Hadoop.

(3)High Flexibility:

XGBoost allow users to define custom optimization objectives and evaluation criteria.
This adds a whole new dimension to the model and there is no limit to what we can do.

(4)Continue on Existing Model:

User can start training an XGBoost model from its last iteration of previous run. This can be of significant advantage in certain specific applications.
GBM implementation of sklearn also has this feature so they are even on this point.

#### XGBoost Parameters

The overall parameters have been divided into 3 categories by XGBoost authors:
1. General Parameters: Guide the overall functioning
2. Booster Parameters: Guide the individual booster (tree/regression) at each step
3. Learning Task Parameters: Guide the optimization performed

##### General Parameters
These define the overall functionality of XGBoost:
1. booster [default=gbtree]
2. silent [default=0]
3. nthread [default to maximum number of threads available if not set]

##### Booster Parameters
Though there are 2 types of boosters, I’ll consider only tree booster here because it always outperforms the linear booster and thus the later is rarely used.
1. eta [default=0.3]
2. min_child_weight [default=1]
3. max_depth [default=6]
4. gamma [default=0]
5. max_leaf_nodes
6. max_delta_step [default=0]
7. subsample [default=1]
8. colsample_bytree [default=1]
9. colsample_bylevel [default=1]
10. alpha [default=0]
11. scale_pos_weight [default=1]


#### Step 1: Fix learning rate and number of estimators for tuning tree-based parameters
In order to decide on boosting parameters, we need to set some initial values of other parameters. Lets take the following values:

1. max_depth = 5 :
 This should be between 3-10. I’ve started with 5 but you can choose a different number as well. 4-6 can be good starting points.
2. min_child_weight = 1 :
 A smaller value is chosen because it is a highly imbalanced class problem and leaf nodes can have smaller size groups.
3. gamma = 0 : 
 A smaller value like 0.1-0.2 can also be chosen for starting. This will anyways be tuned later.
4. subsample, colsample_bytree = 0.8 :
 This is a commonly used used start value. Typical values range between 0.5-0.9.
5. scale_pos_weight = 1:
  Because of high class imbalance.

In [54]:
from xgboost import XGBClassifier
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb1.fit(x_train, y_train)
y_vpred = xgb1.predict(x_validation)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_validation, y_vpred, target_names=target_names))
print(xgb1.score(x_validation, y_validation))

XGBoost:
             precision    recall  f1-score   support

       fake       0.93      0.94      0.94       454
       real       0.94      0.94      0.94       497

avg / total       0.94      0.94      0.94       951

0.9390115667718192


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Step 2: Tune max_depth and min_child_weight
We tune these first as they will have the highest impact on model outcome. To start with, let’s set wider ranges and then we will perform another iteration for smaller ranges.

In [58]:
import numpy as np
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=2, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'max_depth': [4, 5, 6], 'min_child_weight': [4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [59]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96826, std: 0.01209, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.96797, std: 0.01102, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.96829, std: 0.01161, params: {'max_depth': 4, 'min_child_weight': 6},
  mean: 0.96799, std: 0.01151, params: {'max_depth': 5, 'min_child_weight': 4},
  mean: 0.96883, std: 0.01286, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.96808, std: 0.01258, params: {'max_depth': 5, 'min_child_weight': 6},
  mean: 0.96836, std: 0.01062, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.96793, std: 0.01215, params: {'max_depth': 6, 'min_child_weight': 5},
  mean: 0.96882, std: 0.01281, params: {'max_depth': 6, 'min_child_weight': 6}],
 {'max_depth': 5, 'min_child_weight': 5},
 0.9688293188293187)

Here, we have run 12 combinations with wider intervals between values. The ideal values are 5 for max_depth and 5 for min_child_weight. Lets go one step deeper and look for optimum values. We’ll search for values 1 above and below the optimum values because we took an interval of two.

In [60]:
param_test2 = {
    'max_depth':[4,5,6],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=2, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'max_depth': [4, 5, 6], 'min_child_weight': [4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [61]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96826, std: 0.01209, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.96797, std: 0.01102, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.96829, std: 0.01161, params: {'max_depth': 4, 'min_child_weight': 6},
  mean: 0.96799, std: 0.01151, params: {'max_depth': 5, 'min_child_weight': 4},
  mean: 0.96883, std: 0.01286, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.96808, std: 0.01258, params: {'max_depth': 5, 'min_child_weight': 6},
  mean: 0.96836, std: 0.01062, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.96793, std: 0.01215, params: {'max_depth': 6, 'min_child_weight': 5},
  mean: 0.96882, std: 0.01281, params: {'max_depth': 6, 'min_child_weight': 6}],
 {'max_depth': 5, 'min_child_weight': 5},
 0.9688293188293187)

#### Step 3: Tune gamma
Now lets tune gamma value using the parameters already tuned above. Gamma can take various values but I’ll check for 5 values here. You can go into more precise values as.

In [64]:
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [65]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96883, std: 0.01286, params: {'gamma': 0.0},
  mean: 0.96737, std: 0.01144, params: {'gamma': 0.1},
  mean: 0.96635, std: 0.01185, params: {'gamma': 0.2},
  mean: 0.96759, std: 0.01261, params: {'gamma': 0.3},
  mean: 0.96710, std: 0.01297, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.9688293188293187)

#### Step 4: Tune subsample and colsample_bytree
The next step would be try different subsample and colsample_bytree values. Lets do this in 2 stages as well and take values 0.6,0.7,0.8,0.9 for both to start with.

In [69]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=5, gamma=0.0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=140, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [70]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96891, std: 0.01188, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.96693, std: 0.01393, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.96870, std: 0.01024, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.96762, std: 0.01178, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.96830, std: 0.01058, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.96836, std: 0.01502, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.96801, std: 0.01179, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.96622, std: 0.00968, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.96906, std: 0.00866, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.96824, std: 0.01265, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.96883, std: 0.01286, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.96728, std: 0.01017, params: {'colsample_bytree': 0.8, 'subsample'

#### Step 5: Tuning Regularization Parameters

In [72]:
param_test5 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=5, gamma=0.0, subsample=0.6, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=140, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.6),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [73]:
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96906, std: 0.00866, params: {'reg_alpha': 1e-05},
  mean: 0.96797, std: 0.01068, params: {'reg_alpha': 0.01},
  mean: 0.96869, std: 0.01151, params: {'reg_alpha': 0.1},
  mean: 0.96819, std: 0.01018, params: {'reg_alpha': 1},
  mean: 0.50000, std: 0.00000, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.9690576337243005)

In [74]:
param_test6 = {
    'reg_alpha':[0.00001, 0.00005, 0.0001, 0.0005]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=5, gamma=0.0, subsample=0.6, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=140, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.6),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 5e-05, 0.0001, 0.0005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [75]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96906, std: 0.00866, params: {'reg_alpha': 1e-05},
  mean: 0.96906, std: 0.00866, params: {'reg_alpha': 5e-05},
  mean: 0.96906, std: 0.00866, params: {'reg_alpha': 0.0001},
  mean: 0.96919, std: 0.00871, params: {'reg_alpha': 0.0005}],
 {'reg_alpha': 0.0005},
 0.9691895018561685)

In [76]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.0,
        subsample=0.6,
        colsample_bytree=0.8,
        reg_alpha=0.0005,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb2.fit(x_train, y_train)
y_vpred = xgb2.predict(x_validation)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_validation, y_vpred, target_names=target_names))
print(xgb2.score(x_validation, y_validation))

XGBoost:
             precision    recall  f1-score   support

       fake       0.94      0.94      0.94       454
       real       0.94      0.95      0.94       497

avg / total       0.94      0.94      0.94       951

0.9421661409043113


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Step 6: Reducing Learning Rate
Lastly, we should lower the learning rate and add more trees. Lets use the cv function of XGBoost to do the job again.

In [77]:
xgb3 = XGBClassifier(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.0,
        subsample=0.6,
        colsample_bytree=0.8,
        reg_alpha=0.0005,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb3.fit(x_train, y_train)
y_vpred = xgb3.predict(x_validation)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_validation, y_vpred, target_names=target_names))
print(xgb3.score(x_validation, y_validation))

XGBoost:
             precision    recall  f1-score   support

       fake       0.94      0.94      0.94       454
       real       0.95      0.95      0.95       497

avg / total       0.94      0.94      0.94       951

0.9442691903259727


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


##### conclusion for this part
after we reduce the leanring rate and accordingly increased the tree, the accuracy only impove little but longer time cost in the xgb3, so we don't think that xgb3 is better than xgb2

## Final test

In [78]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.0,
        subsample=0.6,
        colsample_bytree=0.8,
        reg_alpha=0.0005,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb2.fit(x_train, y_train)
y_pred = xgb2.predict(x_test)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_test, y_pred, target_names=target_names))
print(xgb2.score(x_test, y_test))

XGBoost:
             precision    recall  f1-score   support

       fake       0.95      0.94      0.94       492
       real       0.94      0.94      0.94       459

avg / total       0.94      0.94      0.94       951

0.9411146161934806


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
